In [ ]:
# En esta celda se puede probar la funcion de analisis de una imagen

from lib.image import image_data
from lib.analysis import dyssynchonia_analysis
import cv2

# Path completo donde esta la imagen (incluyendo nombre y extencion)
path = 'photos_examples/c1d000.tif'

# Ancho de la feta a analisar (es un parametro opcional)
ancho_corte = 5

# Distancia minima en pixeles entre picos
min_dist_between_maxs = 20

image = cv2.imread(path)
# Select ROI
fromCenter = False
showCrosshair = False
seleted_parameters = cv2.selectROI(image, fromCenter, showCrosshair)

# Crop image
x_start = int(seleted_parameters[1])
x_end = x_start + int(seleted_parameters[3])
y_start = int(seleted_parameters[0])
y_end = y_start + int(seleted_parameters[2])

cv2.destroyAllWindows()

image = image_data.get_image_data(image)
image = image_data.crop_vertical(image, x_start, x_end)
image = image_data.crop_horizontal(image, y_start, y_end)

results = dyssynchonia_analysis.analyze_image(image, min_dist_between_maxs, ancho_corte)

# PRUEBA ANALISIS 

In [1]:
# En esta celda se puede probar la funcion de analisis todas las imagenes .tif en un directorio
import os
from lib.image import image_data
from lib.analysis import dyssynchonia_analysis
import cv2

# Ancho de la feta a analisar (es un parametro opcional)
ancho_corte = 5

# Distancia minima en pixeles entre picos
min_dist_between_maxs = 20


# Path donde estan las imagenes
path = "C:/Users/Leand/OneDrive/Documentos/Lean/Analizador_imagenes_calcio/seleccion/050320sel/050320c1"

images_paths = [ f'{path}/{file}' for file in os.listdir(path) if file.endswith(".tif") ]
images = [ cv2.imread(image_path) for image_path in images_paths ]

In [2]:
fromCenter = False
showCrosshair = False

#Tomamos la primer imagen para seleccionar el recorte elegido el resto de las imagenes se van a recortar igual
seleted_parameters = cv2.selectROI(images[0], fromCenter, showCrosshair)

# Crop image
x_start = int(seleted_parameters[1])
x_end = x_start + int(seleted_parameters[3])
y_start = int(seleted_parameters[0])
y_end = y_start + int(seleted_parameters[2])

cv2.destroyAllWindows()

In [3]:
images = [ image_data.get_image_data(image) for image in images ]
images = [ image_data.crop_vertical(image, x_start, x_end) for image in images ]
images = [ image_data.crop_horizontal(image, y_start, y_end) for image in images ]
images

[array([[8, 8, 8, ..., 8, 8, 9],
        [8, 8, 8, ..., 8, 8, 8],
        [8, 8, 8, ..., 8, 8, 8],
        ...,
        [8, 8, 8, ..., 8, 8, 8],
        [8, 8, 8, ..., 8, 8, 8],
        [8, 8, 8, ..., 8, 8, 8]], dtype=uint8),
 array([[13, 13, 13, ..., 12, 12, 13],
        [13, 13, 13, ..., 12, 12, 12],
        [13, 13, 13, ..., 12, 12, 12],
        ...,
        [13, 13, 13, ..., 18, 18, 18],
        [13, 13, 13, ..., 18, 18, 18],
        [13, 13, 13, ..., 18, 18, 18]], dtype=uint8),
 array([[16, 17, 17, ..., 19, 20, 20],
        [16, 17, 17, ..., 19, 19, 19],
        [16, 16, 17, ..., 18, 19, 19],
        ...,
        [12, 13, 13, ..., 13, 13, 13],
        [12, 13, 13, ..., 13, 13, 13],
        [12, 13, 13, ..., 13, 13, 13]], dtype=uint8),
 array([[ 9,  9,  9, ..., 24, 24, 24],
        [ 9,  9,  9, ..., 23, 24, 24],
        [ 9,  9,  9, ..., 23, 23, 23],
        ...,
        [ 8,  8,  9, ..., 15, 15, 15],
        [ 8,  8,  9, ..., 15, 15, 15],
        [ 8,  8,  9, ..., 15, 15, 15]], dt

In [10]:
dy_analysis = [dyssynchonia_analysis.analyze_image(image, min_dist_between_maxs, ancho_corte) for image in images]

In [18]:
with open(path + r'test.txt', 'w') as f:
    f.write(" ".join(map(str, dy_analysis)))

# Analisis de datos
Notas: 
- Image corresponde al análisis de toda la célula y slices corresponde a fetas de esa imagen
- intensidades es el valor que resulta de comprimir (sumando) la matriz con las coordenadas de x,y,z de los pixeles 
- max_peaks_pos es la index de en la lista de intensidades 
- max_peaks_intensities el valor de intensidad que se corresponde con el pico en la posicion analoga de max_peaks_pos

In [5]:
#Imports
import json
import numpy as np
import pandas as pd

In [6]:
#encodes a dictionary into a jason
class NumpyEncoder(json.JSONEncoder):
    """ Special json encoder for numpy types """
    def default(self, obj):
        if isinstance(obj, np.integer):
            return int(obj)
        elif isinstance(obj, np.floating):
            return float(obj)
        elif isinstance(obj, np.ndarray):
            return obj.tolist()
        return json.JSONEncoder.default(self, obj)


def jasonGenerator(path,results):
    """ Generates a .jason file in the path given folder from the a dictionary """
    with open(path + '/analysis_result.jason',"w") as miarch:
        miarch.write(json.dumps(reslust, cls=NumpyEncoder))

In [7]:
def jasonReader(path):
    """ Reads the .jason generated with jasonGenartor the given path"""
    with open( path + '/analysis_result.json','r') as miarch:
        loaded_dict = json.loads(miarch.read())
    return loaded_dict

In [8]:
dicto1 = jasonReader('C:/Users/Leand/OneDrive/Documentos/Lean/Analizador_imagenes_calcio/seleccion/050320sel/050320c1')

In [9]:
#dicto1['slices'][0]
dicto1['image']

{'intensities': [41,
  42,
  44,
  45,
  46,
  47,
  47,
  47,
  47,
  47,
  46,
  46,
  45,
  45,
  44,
  43,
  43,
  42,
  41,
  40,
  40,
  39,
  38,
  37,
  36,
  36,
  35,
  34,
  33,
  33,
  32,
  31,
  31,
  30,
  30,
  29,
  28,
  28,
  27,
  27,
  26,
  26,
  25,
  25,
  25,
  24,
  24,
  23,
  23,
  22,
  22,
  21,
  21,
  21,
  20,
  20,
  19,
  19,
  19,
  19,
  18,
  18,
  18,
  18,
  18,
  18,
  19,
  20,
  22,
  23,
  25,
  26,
  28,
  30,
  33,
  35,
  37,
  39,
  40,
  42,
  44,
  45,
  46,
  47,
  47,
  47,
  47,
  46,
  46,
  46,
  45,
  45,
  44,
  43,
  43,
  42,
  41,
  40,
  40,
  39,
  38,
  37,
  36,
  36,
  35,
  34,
  34,
  33,
  32,
  32,
  31,
  30,
  30,
  29,
  29,
  28,
  28,
  27,
  27,
  26,
  26,
  25,
  25,
  24,
  24,
  23,
  23,
  22,
  22,
  22,
  21,
  21,
  20,
  20,
  20,
  19,
  19,
  18,
  18,
  18,
  17,
  17,
  17,
  18,
  18,
  20,
  21,
  22,
  24,
  26,
  28,
  30,
  32,
  34,
  36,
  38,
  40,
  42,
  44,
  45,
  46,
  47,
  47,
  47,
 